<h1> Neural Net </h1>

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import subprocess
import time
import os
import glob
from torch.utils.data import Dataset, DataLoader
from lucam import Lucam

# --- 기본 설정 (이전과 동일) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
N = 600
LEARNING_RATE_MODEL = 1e-3
LEARNING_RATE_PHASE = 1e-3

# --- 🧠 중간 깊이의 뉴럴 네트워크 모델 정의 ---
class MediumUNetPropagation(nn.Module):
    def __init__(self, in_channels=2, out_channels=1):
        super(MediumUNetPropagation, self).__init__()

        # --- 인코더 (Contracting Path) ---
        # Level 1
        self.enc1 = self.conv_block(in_channels, 64)
        # Level 2
        self.enc2 = self.conv_block(64, 128)
        # Level 3 (추가된 깊이)
        self.enc3 = self.conv_block(128, 256)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # --- 병목 구간 (Bottleneck) ---
        self.bottleneck = self.conv_block(256, 512)

        # --- 디코더 (Expanding Path) ---
        # Level 3
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(256 + 256, 256) # Skip connection 포함
        # Level 2
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(128 + 128, 128) # Skip connection 포함
        # Level 1
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(64 + 64, 64)   # Skip connection 포함

        # --- 최종 출력 레이어 ---
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_c, out_c):
        """두 개의 3x3 Conv와 ReLU, BatchNorm으로 구성된 기본 블록"""
        return nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True)
        )

    def forward(self, phase_map):
        # ‼️ 입력 변환: φ -> [cos(φ), sin(φ)]
        if phase_map.dim() == 3: # (B, H, W) -> (B, 1, H, W)
            phase_map = phase_map.unsqueeze(1)

        x_cos = torch.cos(phase_map)
        x_sin = torch.sin(phase_map)
        x = torch.cat([x_cos, x_sin], dim=1) # (B, 2, N, N)

        # --- 인코더 ---
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))

        # --- 병목 ---
        b = self.bottleneck(self.pool(e3))

        # --- 디코더 + Skip Connections ---
        d3 = self.upconv3(b)
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.dec3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        # --- 출력 ---
        out = self.out_conv(d1)
        return out.squeeze(1) # (B, N, N)

# --- 헬퍼 함수 정의 (이전과 동일) ---
def save_phase_as_image(phase_tensor, filename):
    phase_normalized = (phase_tensor.detach() + torch.pi) / (2 * torch.pi)
    phase_uint8 = (phase_normalized * 255).byte().cpu().numpy()
    Image.fromarray(phase_uint8).save(filename)

def load_and_preprocess_image(path, size=(N, N)):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None: raise FileNotFoundError(f"'{path}' 파일을 찾을 수 없습니다.")
    img = cv2.resize(img, dsize=size)
    img_float = img.astype(np.float32) / np.max(img) # 0~1 사이로 정규화
    return torch.from_numpy(img_float).to('cpu')

def fresnel_cuda(image, lam, dx, z):
    N = 600
    L = N * dx

    x = torch.linspace(-L/2, L/2, N).cuda()
    y = x
    X, Y = torch.meshgrid(x, y)

    k = 2 * torch.pi / lam
    k = k.cuda()

    coeff = torch.exp(1j * k * z) / (1j * lam * z)
    kernel = coeff * torch.exp(1j * k / 2 / z * (X**2 + Y**2))
    transfer = torch.fft.fftshift(torch.fft.fft2(kernel))

    f_image = torch.fft.fftshift(torch.fft.fft2(image))
    f_image = f_image * transfer
    image = torch.fft.ifft2(torch.fft.ifftshift(f_image))
    image = torch.fft.fftshift(image) # Don't know why do this but this should be exist
    return image

# --- 🌟 데이터셋 클래스 정의 🌟 ---
class HolographyDataset(Dataset):
    def __init__(self, image_dir):
        # 이미지 파일 경로 리스트 가져오기
        self.image_paths = glob.glob(os.path.join(image_dir, '*.jpg')) + \
                           glob.glob(os.path.join(image_dir, '*.png'))
                           
        lam = 0.532e-6
        dx = 12.5e-6
        z = 100e-3

        lam = torch.tensor(lam).cuda()
        dx = torch.tensor(dx).cuda()
        z = torch.tensor(z).cuda()

        # 각 이미지에 대한 위상 텐서를 저장할 딕셔너리
        self.phase_tensors = {}
        for path in self.image_paths:
            # 초기 위상은 랜덤으로 생성
            target_intensity = load_and_preprocess_image(path)
            target_amplitude = torch.sqrt(target_intensity).cuda()
            field = fresnel_cuda(target_amplitude, lam, dx, -z)
            phase = torch.angle(field).requires_grad_(True)
            self.phase_tensors[path] = phase
            
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        path = self.image_paths[idx]
        target_intensity = load_and_preprocess_image(path)
        target_amplitude = torch.sqrt(target_intensity)
        phase_tensor = self.phase_tensors[path]
        return target_amplitude.to('cuda'), phase_tensor.to('cuda'), path # 경로도 함께 반환하여 추적

# --- 변수 및 모델 초기화 ---
model = MediumUNetPropagation().to(device)

# 🌟 데이터셋 및 데이터로더 생성
# 'images' 폴더에 학습용 이미지를 넣어주세요.
dataset = HolographyDataset(image_dir='./images')
# 미니배치 크기. GPU 메모리에 따라 조절.
BATCH_SIZE = 1
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# s1, s2 스케일 팩터. 이제 이미지마다 필요할 수 있으나, 우선은 공유
s1 = torch.tensor(1.0, device=device, requires_grad=True)
s2 = torch.tensor(1.0, device=device, requires_grad=True)

# ‼️ 옵티마이저 정의. 이제 위상 텐서는 데이터셋 안에 있으므로, 별도로 최적화
optimizer_model = optim.Adam(list(model.parameters()) + [s2], lr=LEARNING_RATE_MODEL)
# 위상 텐서들을 모아서 phase 옵티마이저에 등록
all_phase_params = list(dataset.phase_tensors.values()) + [s1]
optimizer_phase = optim.Adam(all_phase_params, lr=LEARNING_RATE_PHASE)

loss_fn = torch.nn.MSELoss()

print(f"\n--- 초기화 완료 ---")
print(f"\n--- 초기화 완료 ---")
print(f"총 {len(dataset)}개의 이미지로 데이터셋 구성 완료.")

Using device: cuda

--- 초기화 완료 ---

--- 초기화 완료 ---
총 8개의 이미지로 데이터셋 구성 완료.


In [ ]:
# ‼️ 옵티마이저 정의. 이제 위상 텐서는 데이터셋 안에 있으므로, 별도로 최적화
optimizer_model = optim.Adam(list(model.parameters()) + [s2], lr=1e-4)

In [21]:
from pytorch_msssim import ssim, ms_ssim # Multi-Scale SSIM이 더 성능이 좋을 수 있습니다.
import torch.nn.functional as F

NUM_EPOCHS = 15 # 전체 데이터셋을 몇 번 반복할지
lam = 0.532e-6
dx = 12.5e-6
z = 100e-3

lam = torch.tensor(lam).cuda()
dx = torch.tensor(dx).cuda()
z = torch.tensor(z).cuda()

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*20} Epoch {epoch + 1}/{NUM_EPOCHS} {'='*20}")
    
        # data_loader에서 미니배치 단위로 데이터를 가져옴
    for i, (target_amplitudes, phase_tensors, image_paths) in enumerate(data_loader):
        
        # --- 단계 1: 위상 업데이트 -
        model.eval()
        optimizer_phase.zero_grad()
        
        # U-Net 모델은 배치 입력을 처리할 수 있도록 수정됨
        prediction_for_phase = model(phase_tensors)

        field = torch.exp(1j * prediction_for_phase)
        propagated_field = fresnel_cuda(field, lam, dx, z)
        prediction_for_phase = torch.abs(propagated_field)
        prediction_for_phase = prediction_for_phase/torch.max(prediction_for_phase).item()

        loss_phase = loss_fn(s1 * prediction_for_phase, target_amplitudes)
        loss_phase.backward()
        optimizer_phase.step()
        
        print(f"Epoch {epoch+1}, Batch {i+1} [1/2] 위상 업데이트 완료. Loss: {loss_phase.item():.6f}")

        # --- 단계 2: 모델 업데이트 ---
        # 이 단계에서는 미니배치의 각 이미지에 대해 물리적 실험을 반복해야 함
        
        captured_amplitudes_batch = []
        # 배치 내 각 샘플에 대해 SLM 띄우고 촬영
        for j in range(len(image_paths)):
            phase_to_display = phase_tensors[j]
            
            save_phase_as_image(phase_to_display, 'test.png')
            
            slm_process = subprocess.Popen(['python', 'test.py'])
            time.sleep(2)
            
            # ‼️ 실제 카메라 촬영 로직
            camera = Lucam()
            capture = camera.TakeSnapshot()
            capture = cv2.resize(capture, dsize=(N, N))
            cv2.imwrite('captured_image.png', capture)
            
            slm_process.terminate()
            slm_process.wait()

            captured_amp = torch.sqrt(load_and_preprocess_image('captured_image.png'))
            captured_amplitudes_batch.append(captured_amp)
        
        # 촬영된 이미지들을 하나의 배치 텐서로 결합
        captured_amplitudes = torch.stack(captured_amplitudes_batch)
        
        # 모델 학습
        model.train()
        optimizer_model.zero_grad()
        
        # phase_tensors는 업데이트되었지만, 모델 학습에는 이전 상태를 사용해야 함
        prediction_for_model = model(phase_tensors.detach())

        field = torch.exp(1j * prediction_for_model)
        propagated_field = fresnel_cuda(field, lam, dx, z)
        prediction_for_model = torch.abs(propagated_field)
        prediction_for_model = prediction_for_model / torch.max(prediction_for_model).item()

        loss_model = loss_fn(s2 * prediction_for_model, captured_amplitudes.cuda())
        loss_model.backward()
        optimizer_model.step()
        
        print(f"Epoch {epoch+1}, Batch {i+1} [2/2] 모델 업데이트 완료.")
        temp = phase_tensors.clone()
        output = model(temp.detach()).detach().cpu().numpy()[0]
        output = (output - np.min(output)) / (np.max(output) - np.min(output))
        output = output * 255
        Image.fromarray(output.astype('uint8')).save('output.png')



==================== Epoch 1/15 ====================
Epoch 1, Batch 1 [1/2] 위상 업데이트 완료. Loss: 0.066171
Epoch 1, Batch 1 [2/2] 모델 업데이트 완료.
Epoch 1, Batch 2 [1/2] 위상 업데이트 완료. Loss: 0.138095
Epoch 1, Batch 2 [2/2] 모델 업데이트 완료.
Epoch 1, Batch 3 [1/2] 위상 업데이트 완료. Loss: 0.044803
Epoch 1, Batch 3 [2/2] 모델 업데이트 완료.
Epoch 1, Batch 4 [1/2] 위상 업데이트 완료. Loss: 0.017946
Epoch 1, Batch 4 [2/2] 모델 업데이트 완료.
Epoch 1, Batch 5 [1/2] 위상 업데이트 완료. Loss: 0.078836
Epoch 1, Batch 5 [2/2] 모델 업데이트 완료.
Epoch 1, Batch 6 [1/2] 위상 업데이트 완료. Loss: 0.019213
Epoch 1, Batch 6 [2/2] 모델 업데이트 완료.
Epoch 1, Batch 7 [1/2] 위상 업데이트 완료. Loss: 0.033089
Epoch 1, Batch 7 [2/2] 모델 업데이트 완료.
Epoch 1, Batch 8 [1/2] 위상 업데이트 완료. Loss: 0.049394
Epoch 1, Batch 8 [2/2] 모델 업데이트 완료.

==================== Epoch 2/15 ====================
Epoch 2, Batch 1 [1/2] 위상 업데이트 완료. Loss: 0.038773
Epoch 2, Batch 1 [2/2] 모델 업데이트 완료.
Epoch 2, Batch 2 [1/2] 위상 업데이트 완료. Loss: 0.371331
Epoch 2, Batch 2 [2/2] 모델 업데이트 완료.
Epoch 2, Batch 3 [1/2] 위상 업데이트 완료. Loss: 0

KeyboardInterrupt: 

In [4]:
a = torch.rand((10,10), requires_grad=True)
a

tensor([[0.9336, 0.3582, 0.7935, 0.0520, 0.3762, 0.8230, 0.8240, 0.7717, 0.0079,
         0.2563],
        [0.9462, 0.2194, 0.8983, 0.5424, 0.2604, 0.7501, 0.1395, 0.1190, 0.3600,
         0.1297],
        [0.3989, 0.1994, 0.9521, 0.5441, 0.9915, 0.8992, 0.0383, 0.1927, 0.6817,
         0.8282],
        [0.1613, 0.1447, 0.6904, 0.1973, 0.8128, 0.0421, 0.7949, 0.5161, 0.1538,
         0.4465],
        [0.2018, 0.4343, 0.7569, 0.5306, 0.7159, 0.2957, 0.6866, 0.0284, 0.2571,
         0.9168],
        [0.0722, 0.5894, 0.6778, 0.3464, 0.1135, 0.7081, 0.0522, 0.5975, 0.4147,
         0.4051],
        [0.4297, 0.0296, 0.8246, 0.1053, 0.2656, 0.4284, 0.0789, 0.7853, 0.7905,
         0.3956],
        [0.5438, 0.5039, 0.9148, 0.5283, 0.7852, 0.2799, 0.8295, 0.6605, 0.5125,
         0.3032],
        [0.1538, 0.1275, 0.0366, 0.4837, 0.3076, 0.3640, 0.7123, 0.4583, 0.7257,
         0.0591],
        [0.5978, 0.1666, 0.9902, 0.2688, 0.7349, 0.1601, 0.3599, 0.6982, 0.3267,
         0.9377]], requires_

In [5]:
b = torch.max(torch.abs(a)).item()

In [14]:
import torch

# requires_grad=True로 설정하여 그래디언트 추적
a = torch.ones(2, 3, requires_grad=True)

# 2. .item()을 사용한 경우 (그래디언트 흐름이 끊김)
b = torch.max(torch.abs(a)).item()

c = torch.max(torch.abs(a))
loss = c * 2

# 그래디언트 계산 예시
# c는 연산 그래프에 연결되어 있으므로 역전파 가능
loss.backward()

print("c를 통해 역전파 후 a의 그래디언트:")
print(a.grad) # a에 그래디언트가 계산됨

# b는 파이썬 숫자이므로 역전파의 시작점이 될 수 없음
# 만약 loss = b * 2 라고 한 뒤 loss.backward()를 시도하면 에러 발생

c를 통해 역전파 후 a의 그래디언트:
tensor([[0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333]])
